In [3]:
import numpy as np
import onnxruntime as ort
import onnx
from onnx.checker import check_model
from onnx.tools import update_model_dims
import glob
import pathlib
import warnings
import torch
import json
import pandas as pd
from tqdm import tqdm

# PT Processing

In [23]:
pt_results = pd.read_json('./random_pt.json')

## Conv Errors

In [24]:
pt_results.loc[pt_results['error'] == 1].shape[0]

190

In [25]:
for i, row in enumerate(pt_results.loc[pt_results['error'] == 1]['error_des']):
    print(i, row.split("\n")[-2])
# pt_results.loc[pt_results['error'] == 1]['error_des'][0].split("\n")[-2]

0 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
1 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
2 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
3 RuntimeError: ScalarType ComplexFloat is an unexpected tensor scalar type
4 RuntimeError: ScalarType ComplexFloat is an unexpected tensor scalar type
5 RuntimeError: ScalarType ComplexFloat is an unexpected tensor scalar type
6 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
7 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
8 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
9 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
10 RuntimeError: ScalarType ComplexFloat is an unexpected tensor scalar type
11 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type
12 RuntimeError: ScalarType ComplexDouble is an unexpected tensor scalar type


## Processing

In [26]:
diff_tested_pt = pt_results.loc[(pt_results['error'] == 0) & (pt_results['mean_dist'].notna())]

In [27]:
processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            pt_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                "mad",
            ],
        ]
    ),
    index=diff_tested_pt.index,
)


In [28]:
processed_table

,min_dist,mean_dist,max_dist,std_dev,med_dist
,mad,mad,mad,mad,mad
15_2576916866_pt,NaN,NaN,NaN,NaN,NaN
15_1880911499_pt,NaN,NaN,NaN,NaN,NaN
15_2349686221_pt,NaN,NaN,NaN,NaN,NaN
15_912854280_pt,NaN,NaN,NaN,NaN,NaN
15_4106384593_pt,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
100_3940739593_pt,NaN,NaN,NaN,NaN,NaN
100_2834467258_pt,NaN,NaN,NaN,NaN,NaN
100_4262738135_pt,NaN,NaN,NaN,NaN,NaN


In [29]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = pt_results.loc[label, col[0]][col[1]]

## Incorrect Outputs

In [30]:
mad_median_pt_dist = processed_table['med_dist'][['mad']]
# mad_median_pt_dist = mad_median_pt_dist.join(pt_results[['num_nodes']])
mad_median_pt_dist.loc[mad_median_pt_dist ['mad'] > 1e-3]

,mad
15_3267215162_pt,1.0
15_3007478846_pt,0.965857
15_256668487_pt,6.549249
20_691728509_pt,0.003906
20_1340384958_pt,0.0625
...,...
95_2843366161_pt,51.371769
95_3510495230_pt,2.241486
95_2523235145_pt,1201.538208
95_1393926397_pt,9.868408


## Success

In [31]:
mad_median_pt_dist.loc[mad_median_pt_dist['mad'] <= 1e-3].shape[0]

293

# PT Constrained Processing

In [33]:
pt_results = pd.read_json('./random_pt_con.json')

## Conv Errors

In [34]:
pt_results.loc[pt_results['error'] == 1].shape[0]

0

In [35]:
for i, row in enumerate(pt_results.loc[pt_results['error'] == 1]['error_des']):
    print(i, row.split("\n")[-2])
# pt_results.loc[pt_results['error'] == 1]['error_des'][0].split("\n")[-2]

## Processing

In [36]:
diff_tested_pt = pt_results.loc[(pt_results['error'] == 0) & (pt_results['mean_dist'].notna())]

In [37]:
processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            pt_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                "mad",
            ],
        ]
    ),
    index=diff_tested_pt.index,
)


In [38]:
processed_table

,min_dist,mean_dist,max_dist,std_dev,med_dist
,mad,mad,mad,mad,mad
15_3490005371_pt,NaN,NaN,NaN,NaN,NaN
15_1830730569_pt,NaN,NaN,NaN,NaN,NaN
15_594356564_pt,NaN,NaN,NaN,NaN,NaN
15_3868237654_pt,NaN,NaN,NaN,NaN,NaN
15_3294824347_pt,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
100_2038814902_pt,NaN,NaN,NaN,NaN,NaN
100_723451920_pt,NaN,NaN,NaN,NaN,NaN
100_2738858578_pt,NaN,NaN,NaN,NaN,NaN


In [39]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = pt_results.loc[label, col[0]][col[1]]

## Incorrect Outputs

In [40]:
mad_median_pt_dist = processed_table['med_dist'][['mad']]
# mad_median_pt_dist = mad_median_pt_dist.join(pt_results[['num_nodes']])
mad_median_pt_dist.loc[mad_median_pt_dist ['mad'] > 1e-3]

,mad
15_1830730569_pt,77.951782
15_2701350503_pt,5.415833
15_3707774021_pt,31.257603
15_2664701379_pt,9.655439
15_2859127604_pt,0.265026
...,...
100_4247939447_pt,15001.112305
100_2038814902_pt,6.753756
100_723451920_pt,10.331679
100_2322025169_pt,5.96505


## Success

In [41]:
mad_median_pt_dist.loc[mad_median_pt_dist['mad'] <= 1e-3].shape[0]

1121

# TF Processing

In [56]:
tf_results = pd.read_json('./random_tf.json')

## Conv Errors

In [57]:
tf_results.loc[tf_results['error'] == 1].shape[0]

0

In [58]:
for i, row in enumerate(tf_results.loc[tf_results['error'] == 1]['error_des']):
    print(i, row.split("\n")[-2])
# pt_results.loc[pt_results['error'] == 1]['error_des'][0].split("\n")[-2]

## Processing

In [59]:
diff_tested_tf = tf_results.loc[(tf_results['error'] == 0) & (tf_results['mean_dist'].notna())]

In [60]:
processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            tf_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                "mad",
            ],
        ]
    ),
    index=diff_tested_tf.index,
)


In [61]:
processed_table

,min_dist,mean_dist,max_dist,std_dev,med_dist
,mad,mad,mad,mad,mad
15_2272429665_tf,NaN,NaN,NaN,NaN,NaN
15_2215967651_tf,NaN,NaN,NaN,NaN,NaN
15_2754729144_tf,NaN,NaN,NaN,NaN,NaN
15_2523547174_tf,NaN,NaN,NaN,NaN,NaN
15_360067643_tf,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
95_131659932_tf,NaN,NaN,NaN,NaN,NaN
95_4276798741_tf,NaN,NaN,NaN,NaN,NaN
95_1660442738_tf,NaN,NaN,NaN,NaN,NaN


In [62]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = tf_results.loc[label, col[0]][col[1]]

## Incorrect Outputs

In [63]:
mad_median_tf_dist = processed_table['med_dist'][['mad']]
# mad_median_pt_dist = mad_median_pt_dist.join(pt_results[['num_nodes']])
mad_median_tf_dist.loc[mad_median_tf_dist ['mad'] > 1e-3].shape[0]

37

## Success

In [64]:
mad_median_tf_dist.loc[mad_median_tf_dist['mad'] <= 1e-3].shape[0]

184

# TF Constrained Processing

In [43]:
tf_results = pd.read_json('./random_tf_con.json')

## Conv Errors

In [44]:
tf_results.loc[tf_results['error'] == 1].shape[0]

0

In [45]:
for i, row in enumerate(tf_results.loc[tf_results['error'] == 1]['error_des']):
    print(i, row.split("\n")[-2])
# pt_results.loc[pt_results['error'] == 1]['error_des'][0].split("\n")[-2]

## Processing

In [46]:
diff_tested_tf = tf_results.loc[(tf_results['error'] == 0) & (tf_results['mean_dist'].notna())]

In [47]:
processed_table = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            tf_results[["min_dist", "mean_dist", "max_dist", 'std_dev', "med_dist"]].columns,
            [
                "mad",
            ],
        ]
    ),
    index=diff_tested_tf.index,
)


In [48]:
processed_table

,min_dist,mean_dist,max_dist,std_dev,med_dist
,mad,mad,mad,mad,mad
15_3733555507_tf,NaN,NaN,NaN,NaN,NaN
15_701804835_tf,NaN,NaN,NaN,NaN,NaN
15_1153834586_tf,NaN,NaN,NaN,NaN,NaN
15_1003585824_tf,NaN,NaN,NaN,NaN,NaN
15_3897125719_tf,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
100_41060463_tf,NaN,NaN,NaN,NaN,NaN
100_177588628_tf,NaN,NaN,NaN,NaN,NaN
100_3351817712_tf,NaN,NaN,NaN,NaN,NaN


In [53]:
for label, row in processed_table.iterrows():
    for col, cell in row.iteritems():
        processed_table.loc[label, col] = tf_results.loc[label, col[0]][col[1]]

## Incorrect Outputs

In [55]:
mad_median_tf_dist = processed_table['med_dist'][['mad']]
# mad_median_pt_dist = mad_median_pt_dist.join(pt_results[['num_nodes']])
mad_median_tf_dist.loc[mad_median_tf_dist ['mad'] > 1e-3].shape[0]

31

## Success

In [52]:
mad_median_tf_dist.loc[mad_median_tf_dist['mad'] <= 1e-3].shape[0]

763